Classes

In [22]:
class predicate_gen:
    """Contains a predicate, a description of a part of a current state of a system
    """
    def __init__(self, name: str, args:tuple):
        """Constructor

        Args:
            name (str): name of the predicate
            args (tuple): the keys are the names of the placeholders for each predicate
        Example:
            name: "at", args: ("x", "y" ), negated: False
            
        """        
        self.name = name
        self.args = args
        
    
    def print_predicate(self):
        """Print the predicate

        Returns:
            str: the predicate
        """
        return self.name + "(" + self.args + ")"
    
    def is_equal(self, other):
        """Check if two predicates are equal

        Args:
            other (predicate): the other predicate

        Returns:
            bool: True if the two predicates are equal, False otherwise
        """
        if self.name != other.name:
            return False
        if len(self.args) != len(other.args):
            return False
        for key in self.args:
            if key not in other.args: 
                return False
        return True

In [17]:

class action:
    """Contains an action, a description of an action to be performed on a system
    """
    def __init__(self, name: str, preconditions: list, effects: list, parameters: dict):
        """Constructor

        Args:
            name (str): name of the action
            preconditions (list): list of predicates
            effects (list): list of predicates
            parameters (list): dict of str objects
        
        Example:
            name: "move_on", parameters:["x", "y"]
            preconditions: [{"name": "on_table", negated: False, "args": {"x": None}}, {"name": "clear", negated: False, "args": {"x": None}}, {"name":"clear", negated: False, "args" : {"y": "None}}"], effects: [{"name": "on", negated: False "args": {"x": None, "y": None}}, {"name": "clear", negated: True, "args": {"y": None}}, {"name": "clear", negated: False, "args": {"x": None}}]
        """        
        self.name = name
        self.preconditions = preconditions
        self.effects = effects
        self.parameters = parameters
        self.applied = False # True if the values of the preconditions have values.
    def parameterize_action(self, placeholder_param_dict):
        """Replace the placeholders with the values in the parameter dict

        Args:
            placeholder_param_dict (dict): the keys are the names of the placeholders, the values are str objects
        """
        pass
    def execute_action(self, state, placeholder_param_dict:dict):
        """Execute the action

        Args:
            state (State): the current state of the system
        """
        for effect in self.effects:
            new_pred = predicate(effect.name, effect.args, effect.negated)
            for key in effect.args:
                new_pred.args[key] = placeholder_param_dict[key]
            state.add_predicate(new_pred)

class state:
    """Contains a state, a description of the current state of a system
    """
    def __init__(self, predicates: list):
        """Constructor

        Args:
            predicates (list): list of predicates
        """        
        self.predicates = predicates
    def add_predicate(self, pred):
        """Add a predicate to the state

        Args:
            pred (predicate): the predicate to be added
        """
        # Check if the predicate is already in the state
        for pred_state in self.predicates:
            # If the predicate is already in the state, do nothing
            if pred.is_equal(pred_state):
                return True
            # If the predicate is a negated version of an existing predicate
            elif pred.name == pred_state.name and pred.negated != pred_state.negated:
                pred_state.negated = not pred_state.negated
                return True
        self.predicates.append(pred)
        return True
    def print_state(self):
        """Print the state

        Returns:
            str: the state
        """
        return " ".join([pred.print_predicate() for pred in self.predicates])
    def check_if_conditions_are_met(self, conditions: list):
        """Checks if the conditions are met

        Args:
            conditions (list): list of predicates
        returns:
            empty list if the conditions are met, a list of predicates that are not met otherwise
        """
        conditions_are_met = False
        failed_conditions = []
        for pred in conditions:
            for pred_state in self.predicates:
               if pred.is_equal(pred_state):
                    conditions_are_met = True
                    break
            if not conditions_are_met:
                failed_conditions.append(pred)
            conditions_are_met = False
        return failed_conditions

        
        
          


In [33]:
# class that inherits from predicate_gen, it adds the negated attribute, a method to negate the predicate, and a dictionary that matches the args
# to the arguments of an action
class predicate(predicate_gen):
    """Contains a predicate, a description of a part of a current state of a system
    """
    def __init__(self, name: str, args:tuple, negated: bool, matches: dict):
        """Constructor

        Args:
            name (str): name of the predicate
            args (tuple): the keys are the names of the placeholders for each predicate
            negated (bool): True if the predicate is negated, False otherwise
            matches (dict): the keys are elements of the args tuple. The values are the names of the arguments of the action. 
        """        
        super().__init__(name, args)
        self.negated = negated
        self.matches = matches
    def negate(self):
        """Negate the predicate

        Returns:
            predicate: the negated predicate
        """
        self.negated = not self.negated
    def print_contextual_predicate(self):
        """Print the predicate

        Returns:
            str: the predicate
        """
        return f'{self.name}({self.matches})'
    def is_contextually_equal(self, other):
        """Check if two predicates are equal, taking into account the matches and negation

        Args:
            other (predicate): the other predicate

        Returns:
            bool: True if the two predicates are equal, False otherwise
        """
        if self.name != other.name:
            return False
        if len(self.args) != len(other.args):
            return False
        if self.negated != other.negated:
            return False
        for key in self.matches:
            if key not in other.matches: 
                return False
            else:
                if self.matches[key] != other.matches[key]:
                    return False
        return True

In [36]:
# class called literal that inherits from predicate, it adds a dictionary called values, where each key is a value from the matches dictionary, and the value is a string, a real element from the domain problem
class literal(predicate):
    """Contains a literal, a description of a part of a current state of a system
    """
    def __init__(self, name: str, args:tuple, negated: bool, matches: dict, values: dict):
        """Constructor

        Args:
            name (str): name of the predicate
            args (tuple): the keys are the names of the placeholders for each predicate
            negated (bool): True if the predicate is negated, False otherwise
            matches (dict): the keys are elements of the args tuple. The values are the names of the arguments of the action. 
            values (dict): the keys are the names of the placeholders for each predicate. The values are the values of the placeholders
        """        
        super().__init__(name, args, negated, matches)
        self.values = values
    def print_literal_predicate(self):
        """Print the predicate

        Returns:
            str: the predicate
        """
        # print the name, and then the values of the values dictionary
        return f'{self.name}({self.values})'
        
    def is_literally_equal(self, other):
        """Check if two predicates are equal, taking into account the matches and negation

        Args:
            other (predicate): the other predicate

        Returns:
            bool: True if the two predicates are equal, False otherwise
        """
        if self.name != other.name:
            return False
        if len(self.args) != len(other.args):
            return False
        if self.negated != other.negated:
            return False
        own_comparison_dict = {}
        other_comparison_dict = {}
        for key in self.matches:
            if key not in other.matches: 
                return False
            else:
                # is a mapping betwwen the values of the placeholders and the values of the arguments
                own_comparison_dict[key] = self.values[self.matches[key]]
        for key in other.matches:
            if key not in self.matches: 
                return False
            else:
                # is a mapping betwwen the values of the placeholders and the values of the arguments
                other_comparison_dict[key] = other.values[other.matches[key]]
        for key in own_comparison_dict:
            if own_comparison_dict[key] != other_comparison_dict[key]:
                return False
        return True

In [37]:
gen_predicate1 = predicate_gen("on_table", ("x"))
print(gen_predicate1.print_predicate())
gen_predicate2 = predicate_gen("on_table", ("x"))
context_predicate_1 = predicate("on_table", ("x"), False, {"x": "x"})
print(context_predicate_1.print_contextual_predicate())
context_predicate_2 = predicate("on_table", ("x"), False, {"x": "y"})
print(gen_predicate2.is_equal(gen_predicate1))
print(context_predicate_1.is_contextually_equal(context_predicate_2))
print(context_predicate_1.is_equal(context_predicate_2))

literal1 = literal("on_table", ("x"), False, {"x": "x"}, {"x": "A"})
print(literal1.print_literal_predicate())
literal2 = literal("on_table", ("x"), False, {"x": "x"}, {"x": "B"})
print(literal2.print_literal_predicate())
literal3 = literal("on_table", ("x"), False, {"x": "y"}, {"y": "A"})
print(literal3.print_literal_predicate())
print(literal1.is_literally_equal(literal2))
print(literal1.is_literally_equal(literal3))
print(literal2.is_literally_equal(literal3))


on_table(x)
on_table({'x': 'x'})
True
False
True
on_table({'x': 'A'})
on_table({'x': 'B'})
on_table({'y': 'A'})
False
True
False
